In [1]:
%reload_ext autoreload
%autoreload 2
from cudants.io.image import Image, BatchedImages
from cudants.registration.rigid import RigidRegistration
from cudants.registration.affine import AffineRegistration
import torch
from torch.optim import SGD, Adam
from torch.nn import functional as F
from torch import nn

In [2]:
# img1 = Image.load_file('/data/BRATS2021/training/BraTS2021_00598/BraTS2021_00598_t1.nii.gz')
# img2 = Image.load_file('/data/BRATS2021/training/BraTS2021_00597/BraTS2021_00597_t1.nii.gz')
img1 = Image.load_file('/data/BRATS2021/training/BraTS2021_00262/BraTS2021_00262_t1.nii.gz')
img2 = Image.load_file('/data/BRATS2021/training/BraTS2021_00403/BraTS2021_00403_t1.nii.gz')

# img1 = Image.load_file("images/atlas_2mm_1000_3.nii.gz")
# img2 = Image.load_file("images/atlas_2mm_1001_3.nii.gz")

In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, IntSlider, Layout

def browse_image_slices(image_3d):
    # Infer the dimensions of image
    if len(image_3d.shape) == 3:
        dim_x, dim_y, dim_z = image_3d.shape
        color = False
    else:
        dim_x, dim_y, dim_z = image_3d.shape[:-1]
        color = True
    # Define a function to visualize the slices
    def plot_slice(x, y, z):
        fig, ax = plt.subplots(1, 3, figsize=(15, 5))

        ax[0].imshow(image_3d[x, :, :], cmap='gray' if not color else None)
        ax[0].set_title('Slice at X = %d' % x)

        ax[1].imshow(image_3d[:, y, :], cmap='gray' if not color else None)
        ax[1].set_title('Slice at Y = %d' % y)

        ax[2].imshow(image_3d[:, :, z], cmap='gray' if not color else None)
        ax[2].set_title('Slice at Z = %d' % z)

        plt.show()

    # Use interact to create the UI with sliders
    interact(plot_slice,
             x=IntSlider(min=0, max=dim_x-1, value=dim_x//2, layout=Layout(width='600px')),
             y=IntSlider(min=0, max=dim_y-1, value=dim_y//2, layout=Layout(width='600px')),
             z=IntSlider(min=0, max=dim_z-1, value=dim_z//2, layout=Layout(width='600px')))

In [4]:
# Test the function
browse_image_slices(img1.array[0, 0].data.cpu().numpy())

interactive(children=(IntSlider(value=77, description='x', layout=Layout(width='600px'), max=154), IntSlider(v…

In [5]:
# Test the function
browse_image_slices(img2.array[0, 0].data.cpu().numpy())

interactive(children=(IntSlider(value=77, description='x', layout=Layout(width='600px'), max=154), IntSlider(v…

In [6]:
from cudants.registration.affine import AffineRegistration
fixed = BatchedImages([img1,])
moving = BatchedImages([img2,])
# transform = AffineRegistration([8, 4, 2, 1], [1000, 500, 250, 100], fixed, moving, \
transform = AffineRegistration([8, 4, 2, 1], [200, 100, 50, 20], fixed, moving, \
    loss_type='cc', optimizer='Adam', optimizer_lr=3e-4, optimizer_momentum=0.9)
moved = transform.optimize(save_transformed=True)

/home/rohitrango/code/CUDANTs/cudants/utils/imageutils.py:77: UserWarning: operator() sees varying value in profiling, ignoring and this should be handled by GUARD logic (Triggered internally at ../torch/csrc/jit/codegen/cuda/parser.cpp:3668.)
  image_smooth = separable_filtering(image, gaussians)
/home/rohitrango/code/CUDANTs/cudants/registration/affine.py:76: UserWarning: operator() profile_node %70 : int[] = prim::profile_ivalue[profile_failed="varying profile values"](%s.1)
 does not have profile information (Triggered internally at ../torch/csrc/jit/codegen/cuda/graph_fuser.cpp:105.)
  moving_image_blur = separable_filtering(moving_arrays, gaussians)
scale: 1, iter: 19/20, loss: -0.042983: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 24.84it/s]


In [7]:
browse_image_slices(moved[-1][0, 0].data.cpu().numpy() - img1.array[0, 0].data.cpu().numpy())
browse_image_slices(moved[-1][0, 0].data.cpu().numpy())
browse_image_slices(img1.array[0, 0].data.cpu().numpy())

interactive(children=(IntSlider(value=77, description='x', layout=Layout(width='600px'), max=154), IntSlider(v…

interactive(children=(IntSlider(value=77, description='x', layout=Layout(width='600px'), max=154), IntSlider(v…

interactive(children=(IntSlider(value=77, description='x', layout=Layout(width='600px'), max=154), IntSlider(v…

In [8]:
browse_image_slices(moved[-1][0, 0].data.cpu().numpy() - img2.array[0, 0].data.cpu().numpy())

interactive(children=(IntSlider(value=77, description='x', layout=Layout(width='600px'), max=154), IntSlider(v…

In [25]:
from cudants.registration.logdemons import LogDemonsRegistration
reg = LogDemonsRegistration([4, 2, 1], [50, 50, 50], fixed, moving, 
# reg = LogDemonsRegistration([1], [100], fixed, moving,
                eps_prime=0.5, init_affine=transform.get_affine_matrix(),  
                optical_flow_method='gauss-newton', update_sigma=0.25,
                                optical_flow_sigma=0.5, use_lie_bracket=False)
saved_deformable = reg.optimize(save_transformed=True)

scale: 1, iter: 49/50, loss: 218.361786: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 45.09it/s]


In [26]:
transform.get_affine_matrix()

tensor([[[ 1.0263,  0.0218, -0.0113, -0.0222],
         [-0.0560,  0.9364,  0.0116, -0.0421],
         [ 0.0218, -0.0467,  0.9354,  0.0076],
         [ 0.0000,  0.0000,  0.0000,  1.0000]]], device='cuda:0')

In [27]:
# browse_image_slices(saved_deformable[-1][0, 0].data.cpu().numpy()- moved[-1][0, 0].data.cpu().numpy())
browse_image_slices(saved_deformable[-1][0, 0].data.cpu().numpy())
browse_image_slices(img1.array[0, 0].data.cpu().numpy())
browse_image_slices(img2.array[0, 0].data.cpu().numpy())
# browse_image_slices(moved[-1][0, 0].data.cpu().numpy())

interactive(children=(IntSlider(value=77, description='x', layout=Layout(width='600px'), max=154), IntSlider(v…

interactive(children=(IntSlider(value=77, description='x', layout=Layout(width='600px'), max=154), IntSlider(v…

interactive(children=(IntSlider(value=77, description='x', layout=Layout(width='600px'), max=154), IntSlider(v…

## Check Optical Flow

In [12]:
from cudants.utils.opticalflow import OpticalFlow

In [13]:
of = OpticalFlow(sigma=0.5)

In [14]:
velfield = of(fixed(), moving())

In [15]:
of._method_name

'gauss-newton'

In [16]:
browse_image_slices(velfield[0].permute(1, 2, 3, 0).data.cpu().numpy() - 1*velfield.min().data.cpu().numpy())

interactive(children=(IntSlider(value=77, description='x', layout=Layout(width='600px'), max=154), IntSlider(v…

In [17]:
velfield.max(), velfield.min()

(tensor(0.4760, device='cuda:0'), tensor(-0.4740, device='cuda:0'))

In [18]:
from cudants.utils.imageutils import image_gradient

In [19]:
fixed().shape

torch.Size([1, 1, 155, 240, 240])

In [20]:
img = fixed()
grad = image_gradient(img)
print(grad.min(), grad.max())

tensor(-365.5000, device='cuda:0') tensor(377., device='cuda:0')


In [21]:
grad[0].flatten(1).min(1).values, grad[0].flatten(1).max(1).values

(tensor([-365.5000, -365.5000, -365.5000], device='cuda:0'),
 tensor([377., 377., 377.], device='cuda:0'))

In [22]:
browse_image_slices(grad[0, 2].data.cpu().numpy())

interactive(children=(IntSlider(value=77, description='x', layout=Layout(width='600px'), max=154), IntSlider(v…

In [23]:
from cudants.losses.cc import gaussian_1d, separable_filtering
kernel = gaussian_1d(torch.tensor(2, device=fixed.device))

In [24]:
blurred = separable_filtering(fixed(), kernel)

In [ ]:
browse_image_slices(blurred[0, 0].data.cpu().numpy())
browse_image_slices(fixed()[0, 0].data.cpu().numpy())